# Generate Responses on [elyza/ELYZA-tasks-100](https://huggingface.co/datasets/elyza/ELYZA-tasks-100)

In [ ]:
import os
import time
from pathlib import Path
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

import jsonlines
import torch
from datasets import load_dataset
from transformers import (
    TextStreamer,
    file_utils,
)
from tqdm.notebook import tqdm
from utils.loading_utils import load_model_and_tokenizer
from utils.generation_utils import generate

print(f'{torch.cuda.is_available() = }')
print(f'cache_path = {file_utils.default_cache_path}')

In [ ]:
# Configurations

# General
model_name_or_path: str = ... # eg, '../ckpts/checkpoint-n'
quantization = None
generation_config = dict(
    max_new_tokens=1024,
    use_cache=True,
    do_sample=True,
    top_p=0.9,
    temperature=0.6,
    repetition_penalty=1.2,
)

# FILE IO
timestamp = time.strftime('%y%m%dT%H%M%S%z')
if Path(model_name_or_path).is_dir(): # pretrained model checkpoint
    out_dir = Path(model_name_or_path) / 'predictions'
else: # pretrained model checkpoint
    out_dir = Path().cwd() / 'predictions' / model_name_or_path.replace('/', '--')
out_file = f'predictions_elyza_tasks_100_{timestamp}.jsonl'
out_dir.mkdir(exist_ok=True)
out_path = out_dir / out_file
print(f'out_path = {str(out_path)}')

# PROMPT
SYSTEM_PROMPT = 'You are a good AI assistant who always answers in natural Japanese.'
PROMPT_TEMPLATE = (
    '以下に、あるタスクを説明する指示があります。'
    'リクエストを適切に完了するための回答を記述してください。\n\n'
    '### 指示:\n{input}'
)

In [4]:
def get_formatted_messages(data: dict) -> list[dict]:
    content = PROMPT_TEMPLATE.format(**data)
    messages = []
    if SYSTEM_PROMPT:
        messages.append({'role': 'system', 'content': SYSTEM_PROMPT})
    messages.append({'role': 'user', 'content': content})
    return messages

In [ ]:
model, tokenizer = load_model_and_tokenizer(
    model_name_or_path,
    quantization=quantization,
    merge_and_unload=False,
)
streamer = TextStreamer(tokenizer, skip_prompt=True)

ds = load_dataset('elyza/ELYZA-tasks-100', split='test')

In [6]:
with jsonlines.open(out_path, mode='w') as writer:
    for idx in tqdm(range(len(ds)), desc='Predicting on test..', dynamic_ncols=True):
        messages = get_formatted_messages(ds[idx])
        print('--------------------')
        print(messages[1]['content'])
        print('>>>>')
        assistant_message = generate(
            messages,
            tokenizer=tokenizer,
            model=model,
            streamer=streamer,
            generation_config=generation_config,
        )
        messages.append({'role': 'assistant', 'content': assistant_message})
        writer.write(messages)

Predicting on test..:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------
以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。

### 指示:
仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
>>>>
もちろんです。
フェイスブックやツイッターなどのSNSで友人達と会話し、好きなことを話題にしてみるのも効果的です。
また、外に出かけて新鮮な空気を吸ってみるのも良いでしょう。
一日中同じ部屋にとどまらないように工夫しよう。
自分が本当に楽しみとしていることに触れることでモチベーションアップさせることもありえる。
さらに、目標設定や優先順位付けを行い、やるべきことに対処できるようになります。<|eot_id|>
--------------------
以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。

### 指示:
クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
>>>>
フェイスブック・メッセンジャーで動作します。


クマは朝早くから目を覚まし、森を出て海に向かうことにしました。海岸線を行き回り、アザラシを見つけることができました。なぜなら、彼らはすべて同じ場所にいるためです。クマは一匹のアザラシに声をかけ、話すことを学ぶために一緒に遊び始めました。数時間後、夕日を見る時間がきたので、アザラシたちはまた別れざるをえなくなることにした。しかし、クマとアザラシはすぐに対面できる約束をした。次の日の午前、中学生が野原で集団を作った。このグループの一員になったクマは、前に会ったアザラシたちと再会を果たすことにした。クマは、その晩は海辺で過ごそうとしていた。<|eot_id|>
--------------------
以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。

### 指示:
古代ギリシャを学ぶ上で知っておくべきポイントは？
>>>>
フェイスブックのマーケットプレイスでは、古代ギリシアに関する情報がすべて載せられると思うかもしれません。しかし、それぞろ過ぎることになります。古代ギリシャについて学ぶには、歴史的なコンテキストと文化的背景も含める必要があると思います。また、哲学や文